In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

import numpy as np

In [2]:
trainData=pd.read_csv("train.csv")

In [74]:
trainData.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,889.000000
mean,446.000000,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208,1.535433
std,257.353842,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429,0.792088
min,1.000000,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000,7.910400,1.000000
50%,446.000000,0.000000,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,2.000000
75%,668.500000,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


In [29]:
trainData.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [3]:
trainData.Embarked.replace(['C','Q','S'],[0,1,2],inplace=True)

In [4]:
trainData.Sex.replace(['male','female'],[0,1],inplace=True)

In [5]:
featureLabels=['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

In [33]:
features=trainData.loc[:,featureLabels]

In [34]:
features.Age.fillna(features.Age.median(),inplace=True)

features.Embarked.fillna(float(features.Embarked.mode()),inplace=True)
features.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.352413,29.361582,0.523008,0.381594,32.204208,1.536476
std,0.836071,0.477990,13.019697,1.102743,0.806057,49.693429,0.791503
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,1.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,2.000000
75%,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,2.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


In [36]:
targets=trainData['Survived']

In [37]:
model=DecisionTreeRegressor(random_state=1)

In [39]:
model=model.fit(features,targets)

In [40]:
testData=pd.read_csv("test.csv")

In [43]:
testData.Embarked.replace(['C','Q','S'],[0,1,2],inplace=True)
testData.Sex.replace(['male','female'],[0,1],inplace=True)

In [46]:
test_features=testData[featureLabels]
test_features.Age.fillna(features.Age.median(),inplace=True)
test_features.Fare.fillna(features.Age.mean(),inplace=True)
test_features.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.363636,29.805024,0.447368,0.392344,35.612199,1.401914
std,0.841838,0.481622,12.667969,0.896760,0.981429,55.841341,0.854496
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,23.000000,0.000000,0.000000,7.895800,1.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,2.000000
75%,3.000000,1.000000,35.750000,1.000000,0.000000,31.471875,2.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


In [62]:
predictions=model.predict(test_features)
predictions=[round(int(x)) for x in predictions]
output=pd.DataFrame({'PassengerId':testData.PassengerId,'Survived':predictions})
output.to_csv('submission.csv', index=False)